In [ ]:
import tensorflow as tf
from tensorflow.python.keras.applications.vgg19 import VGG19 # importing the model.
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.applications.vgg19 import preprocess_input
from tensorflow.python.keras.models import Model
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
model = VGG19(include_top = False,weights = 'imagenet')# we don't need the top layer and the weights of imagenet are assigned
model.trainable = False # We only want to apply the pre_trained weights to our inputs instead of training the model with the inputs.
model.summary()

In [ ]:
def load_and_process_image(image_path):
  img=load_img(image_path)
  img=img_to_array(img)
  img=preprocess_input(img)
  img=np.expand_dims(img, axis = 0) # because the model expects as 4-dim input
  return img

In [ ]:
def deprocess(x):
  # perform the inverse of the preprocessiing step
  x[:,:,0] += 103.939
  x[:,:,1] += 116.779
  x[:,:,2] += 123.68
  x = x[:,:,::-1] # reversing the order of channels.
  x = np.clip(x, 0, 255).astype('uint8') # 0-255 is the range of pixels values
  return x

In [ ]:
def display_image(image,save=False):
  if len(image.shape) == 4:
      img = np.squeeze(image, axis = 0) # reducing 4-dim to 3-dim
  img = deprocess(img)
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img)
  if save==True:
    plt.savefig('Output.png')

In [ ]:
def load_and_save_img(path,con=True):
  img=cv2.imread(path)
  res=cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
  if(con):
    cv2.imwrite('ContentImg.jpg',res)
  else:
    cv2.imwrite('StyleImg.jpg',res)

In [ ]:
style_layers = ['block1_conv1','block1_conv2','block2_conv1','block2_conv2','block3_conv1'] # layers from which features of style img are extracted 
content_layer = 'block4_conv4'# layers from which features of content img are extracted
# intermediate models
content_model = Model(inputs = model.input,outputs = model.get_layer(content_layer).output)
style_models = [Model(inputs = model.input,outputs = model.get_layer(layer).output) for layer in style_layers]

In [ ]:
# Content Cost
def content_cost(content, generated):
  a_C = content_model(content)
  a_G = content_model(generated)
  cost = tf.reduce_mean(tf.square(a_C - a_G))
  return cost

In [ ]:
def gram_matrix(A):
  channels = int(A.shape[-1])
  a = tf.reshape(A, [-1, channels]) # shape from x,y,c to x*y,c
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a = True) #a.aT
  return gram / tf.cast(n, tf.float32)# scaling down using no of elements n

In [ ]:
style_layer_wts = [1.0, 0.8, 0.1, 0.1, 0.2]
def style_cost(style, generated):
  J_style = 0    
  for i,style_model in enumerate(style_models):
    a_S = style_model(style)
    a_G = style_model(generated)
    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)
    current_cost = tf.reduce_mean(tf.square(GS - GG))*style_layer_wts[i]
    J_style += current_cost 
  return (J_style/(224*224*len(style_models)))

In [ ]:
generated_images = []
costs=[]
def training_loop(content_path, style_path, iterations = 10, a = 30., b = 10.):
  # initialise
  content = load_and_process_image(content_path)
  style = load_and_process_image(style_path)
  generated = tf.Variable(content, dtype = tf.float32)    
  opt = tf.optimizers.Adam(learning_rate = 10.) 
  for i in range(iterations):
    with tf.GradientTape() as tape:
      J_content = content_cost(content, generated)
      J_style = style_cost(style, generated)
      J_total = a * J_content + b * J_style
    grads = tape.gradient(J_total, generated)
    opt.apply_gradients([(grads, generated)])
    costs.append(J_total.numpy())
    if i%50==0:
      display_image(generated.numpy())
      generated_images.append(generated.numpy())
      print("Iteration:{}, Total Cost:{}".format(i+1,J_total))

In [ ]:
load_and_save_img('content.jpg',True)
load_and_save_img('style.jpg',False)
img = load_and_process_image('ContentImg.jpg')
display_image(img)

In [ ]:
iterations=200
training_loop('ContentImg.jpg','StyleImg.jpg',iterations)

In [ ]:
plt.plot(range(iterations), costs)
plt.xlabel("Iterations")
plt.ylabel("Total Cost")
plt.show()
display_image(generated_images[-1],True)